<a href="https://colab.research.google.com/gist/maclandrol/70b320110559418e964e04badb1d361e/15_IA_Radiologie_Synthese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intelligence Artificielle en Radiologie : Classification de Rapports de Radiographies Thoraciques

## 🎯 Objectifs d'apprentissage

Dans ce notebook pratique, vous allez découvrir comment l'intelligence artificielle peut aider en radiologie en apprenant à :

1. **Analyser des rapports de radiologie** - Comprendre comment les rapports de radiographies thoraciques sont structurés
2. **Utiliser le traitement du langage naturel (NLP)** - Appliquer l'IA pour analyser du texte médical
3. **Construire un classificateur médical** - Créer un modèle qui peut distinguer les rapports normaux des rapports anormaux
4. **Tester votre modèle** - Uploader et tester de nouveaux rapports

## 🏥 Contexte médical

En tant qu'étudiant en médecine, vous savez que l'interprétation des radiographies thoraciques est une compétence fondamentale. Ce notebook vous montrera comment l'IA peut assister (mais jamais remplacer !) le diagnostic médical.

**Données utilisées** : Rapports de radiographies thoraciques du dataset de l'Université d'Indiana, disponible via le service Open-i de la National Library of Medicine.

---

### ⚠️ Important - Considérations éthiques

L'IA en médecine doit toujours être utilisée comme un **outil d'assistance**, jamais comme un remplacement du jugement clinique. Ce notebook est à des fins éducatives uniquement.

## 📚 Installation des bibliothèques nécessaires

Nous utiliserons :
- **fastai** : Bibliothèque de deep learning simplifiée
- **pandas** : Pour la manipulation des données
- **matplotlib** : Pour la visualisation
- **scikit-learn** : Pour les métriques d'évaluation

In [ ]:
# Installation des bibliothèques nécessaires (uniquement pour Google Colab)
try:
    import google.colab
    print("🔧 Installation des bibliothèques pour Google Colab...")
    !pip install fastai -q
    !pip install datasets -q
    print("✅ Installation terminée !")
except ImportError:
    print("💻 Environnement local détecté")

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Fastai pour le deep learning
from fastai.text.all import *
from fastai.text import *

# Pour le téléchargement des fichiers
import requests
import zipfile
import io

print("📦 Toutes les bibliothèques ont été importées avec succès !")

## 🔍 Chargement et exploration des données

### Comprendre les données médicales

Nous allons travailler avec des rapports de radiographies thoraciques. Ces rapports contiennent :
- **Le texte du rapport** : Description de ce que voit le radiologue
- **L'étiquette** : Normal vs Anormal (basé sur les termes MeSH)

**Rappel anatomique** : Une radiographie thoracique normale devrait montrer :
- Silhouette cardiaque normale
- Champs pulmonaires clairs
- Absence d'épanchement pleural
- Absence de pneumothorax

In [ ]:
# Création d'un dataset d'exemple pour la démonstration
# En réalité, vous utiliseriez le vrai dataset d'Indiana University

# Exemples de rapports normaux
rapports_normaux = [
    "La silhouette cardiaque et la taille du médiastin sont dans les limites normales. Il n'y a pas d'œdème pulmonaire. Il n'y a pas de consolidation focale. Il n'y a pas de signe d'épanchement pleural. Il n'y a pas d'évidence de pneumothorax.",
    "La silhouette cardiomédiastinale est dans les limites normales pour la taille et le contour. Les poumons sont normalement gonflés sans évidence de maladie de l'espace aérien focal, d'épanchement pleural, ou de pneumothorax.",
    "Les poumons apparaissent clairs. Les volumes pulmonaires sont normaux. Le cœur et les vaisseaux pulmonaires apparaissent normaux. Les espaces pleuraux sont clairs.",
    "Radiographie thoracique normale. Pas d'anomalie cardiopulmonaire aiguë.",
    "Cœur de taille normale. Poumons clairs bilatéralement. Pas de pneumothorax ni d'épanchement pleural."
]

# Exemples de rapports anormaux
rapports_anormaux = [
    "Il y a une opacité accrue dans le lobe supérieur droit avec une masse possible et une zone associée d'atélectasie ou de consolidation focale. Recommandation d'un scanner thoracique pour évaluation supplémentaire.",
    "Cardiomégalie modérée. Œdème pulmonaire bilatéral. Épanchements pleuraux bilatéraux, plus important à droite.",
    "Pneumothorax spontané du côté droit avec collapsus pulmonaire partiel. Drainage thoracique recommandé.",
    "Consolidation du lobe inférieur gauche compatible avec une pneumonie. Épanchement pleural associé.",
    "Multiples nodules pulmonaires bilatéraux. Élargissement hilaire bilatéral. Suspicion de processus malin ou infectieux."
]

# Création du DataFrame
data = {
    'rapport': rapports_normaux + rapports_anormaux,
    'étiquette': ['normal'] * len(rapports_normaux) + ['anormal'] * len(rapports_anormaux)
}

df = pd.DataFrame(data)
print(f"📊 Dataset créé avec {len(df)} rapports")
print(f"   - {len(rapports_normaux)} rapports normaux")
print(f"   - {len(rapports_anormaux)} rapports anormaux")

# Affichage des premières lignes
print("\n🔍 Aperçu des données :")
df.head()

In [ ]:
# Exploration des données
print("📈 Distribution des étiquettes :")
print(df['étiquette'].value_counts())

# Visualisation
plt.figure(figsize=(8, 5))
df['étiquette'].value_counts().plot(kind='bar', color=['lightgreen', 'lightcoral'])
plt.title('Distribution des Rapports : Normal vs Anormal')
plt.xlabel('Type de Rapport')
plt.ylabel('Nombre de Rapports')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n💡 Analyse des longueurs de texte :")
df['longueur'] = df['rapport'].str.len()
print(f"Longueur moyenne : {df['longueur'].mean():.1f} caractères")
print(f"Longueur médiane : {df['longueur'].median():.1f} caractères")

## 👀 Exemples de rapports pour comprendre les différences

### 🟢 Exemple de rapport NORMAL
Regardez les termes clés qui indiquent un examen normal :

In [ ]:
# Affichage d'un exemple de rapport normal
rapport_normal = df[df['étiquette'] == 'normal'].iloc[0]['rapport']
print("🟢 RAPPORT NORMAL :")
print("=" * 50)
print(rapport_normal)
print("=" * 50)

print("\n🔍 Mots-clés typiques d'un rapport normal :")
mots_normaux = ['limites normales', 'clairs', 'pas d\'', 'normale', 'normalement']
for mot in mots_normaux:
    if mot.lower() in rapport_normal.lower():
        print(f"   ✅ '{mot}' trouvé dans le rapport")

In [ ]:
# Affichage d'un exemple de rapport anormal
rapport_anormal = df[df['étiquette'] == 'anormal'].iloc[0]['rapport']
print("🔴 RAPPORT ANORMAL :")
print("=" * 50)
print(rapport_anormal)
print("=" * 50)

print("\n🔍 Mots-clés typiques d'un rapport anormal :")
mots_anormaux = ['opacité', 'masse', 'consolidation', 'atélectasie', 'recommandation']
for mot in mots_anormaux:
    if mot.lower() in rapport_anormal.lower():
        print(f"   ⚠️ '{mot}' trouvé dans le rapport")

## 🔧 Préparation des données pour l'IA

### Comprendre le préprocessing en NLP médical

Avant d'entraîner notre modèle, nous devons :
1. **Tokenizer** : Diviser le texte en mots/tokens
2. **Numériser** : Convertir les mots en nombres que l'IA peut comprendre
3. **Créer des DataLoaders** : Préparer les données pour l'entraînement

In [ ]:
# Préparation des données pour fastai
print("🔧 Préparation des données...")

# Division en train/validation (80/20)
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['étiquette'], random_state=42)

print(f"📊 Données d'entraînement : {len(train_df)} rapports")
print(f"📊 Données de validation : {len(valid_df)} rapports")

# Création des DataLoaders avec fastai
dls = TextDataLoaders.from_df(
    df, 
    text_col='rapport', 
    label_col='étiquette',
    valid_pct=0.2,
    seed=42,
    bs=2  # Petit batch size pour la démo
)

print("✅ DataLoaders créés avec succès !")
print(f"Classes détectées : {dls.vocab}")

In [ ]:
# Visualisation d'un batch de données
print("🔍 Exemple de batch de données préparées :")
dls.show_batch(max_n=3)

## 🤖 Construction et entraînement du modèle

### Comprendre l'architecture du modèle

Nous utilisons un **modèle de langage pré-entraîné** (transfer learning) :
1. **Base** : Un modèle déjà entraîné sur beaucoup de texte
2. **Fine-tuning** : Adaptation spécifique à nos rapports médicaux
3. **Classification** : Ajout d'une couche pour prédire normal/anormal

### Pourquoi cette approche fonctionne en médecine ?
- Les modèles pré-entraînés comprennent déjà la langue
- Nous les spécialisons pour le vocabulaire médical
- Moins de données nécessaires qu'un entraînement from scratch

In [ ]:
print("🤖 Création du modèle de classification...")

# Création du learner avec un modèle pré-entraîné
learn = text_classifier_learner(
    dls, 
    AWD_LSTM,  # Architecture de réseau de neurones
    metrics=accuracy,  # Métrique d'évaluation
    drop_mult=0.5  # Régularisation pour éviter le surajustement
)

print("✅ Modèle créé avec succès !")
print("\n📋 Détails du modèle :")
print(f"   - Architecture : AWD-LSTM (adapté pour le texte médical)")
print(f"   - Classes à prédire : {dls.vocab}")
print(f"   - Métrique d'évaluation : Accuracy")

In [ ]:
# Recherche du taux d'apprentissage optimal
print("📈 Recherche du taux d'apprentissage optimal...")
learn.lr_find()

In [ ]:
# Entraînement du modèle
print("🏃‍♂️ Début de l'entraînement...")
print("(Cela peut prendre quelques minutes)")

# Entraînement avec fine-tuning progressif
learn.fine_tune(
    epochs=3,  # Nombre d'époques
    base_lr=2e-3  # Taux d'apprentissage
)

print("✅ Entraînement terminé !")

## 📊 Évaluation du modèle

### Métriques importantes en médecine

En médecine, il est crucial d'évaluer :
- **Sensibilité** : Capacité à détecter les cas anormaux (éviter les faux négatifs)
- **Spécificité** : Capacité à identifier les cas normaux (éviter les faux positifs)
- **Accuracy** : Précision globale

**Question clinique** : Préférez-vous un modèle très sensible (détecte tous les problèmes mais avec des fausses alarmes) ou très spécifique (moins de fausses alarmes mais risque de manquer des problèmes) ?

In [ ]:
# Évaluation détaillée du modèle
print("📊 Évaluation du modèle...")

# Matrice de confusion
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(6, 6))
plt.title('Matrice de Confusion\n(Ligne = Réel, Colonne = Prédit)')
plt.show()

# Métriques détaillées
from sklearn.metrics import classification_report

# Obtention des prédictions
preds, targets = learn.get_preds()
predicted_labels = preds.argmax(dim=1)

# Conversion en noms de classes
class_names = dls.vocab
y_true = [class_names[i] for i in targets]
y_pred = [class_names[i] for i in predicted_labels]

print("\n📈 Rapport de classification détaillé :")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Analyse des erreurs
print("🔍 Analyse des erreurs du modèle :")
print("(Quels rapports le modèle a-t-il mal classifiés ?)")

# Afficher les erreurs avec les plus grandes pertes
interp.plot_top_losses(6, nrows=3, figsize=(12, 8))
plt.tight_layout()
plt.show()

## 🧪 Test du modèle avec de nouveaux rapports

### Zone de test interactive

Maintenant, testons notre modèle avec de nouveaux rapports ! Vous pouvez :
1. Utiliser les exemples fournis
2. Saisir vos propres rapports
3. Voir la confiance du modèle dans ses prédictions

In [ ]:
# Fonction pour tester un rapport
def tester_rapport(texte_rapport):
    """Teste un rapport et affiche la prédiction avec confiance"""
    print(f"📝 Rapport à analyser :")
    print(f"'{texte_rapport}'")
    print("\n" + "="*50)
    
    # Prédiction
    pred_class, pred_idx, probs = learn.predict(texte_rapport)
    
    # Affichage des résultats
    confidence = probs[pred_idx].item() * 100
    
    if pred_class == 'normal':
        emoji = "🟢"
        couleur = "NORMAL"
    else:
        emoji = "🔴"
        couleur = "ANORMAL"
    
    print(f"🤖 Prédiction du modèle : {emoji} {couleur}")
    print(f"🎯 Confiance : {confidence:.1f}%")
    
    # Probabilités détaillées
    print(f"\n📊 Probabilités détaillées :")
    for i, classe in enumerate(learn.dls.vocab):
        prob = probs[i].item() * 100
        print(f"   {classe:>8} : {prob:5.1f}%")
    
    print("\n" + "="*50 + "\n")

# Tests avec des exemples
print("🧪 TESTS DU MODÈLE")
print("Testons notre modèle avec différents rapports...\n")

# Exemple 1 : Rapport clairement normal
rapport_test_1 = "Cœur de taille normale. Poumons clairs sans infiltrat. Pas d'épanchement pleural. Radiographie normale."
tester_rapport(rapport_test_1)

# Exemple 2 : Rapport clairement anormal
rapport_test_2 = "Pneumonie du lobe inférieur droit avec consolidation importante. Épanchement pleural modéré. Cardiomégalie."
tester_rapport(rapport_test_2)

## 📤 Zone de test personnalisée

### Testez vos propres rapports !

Utilisez la zone ci-dessous pour tester vos propres rapports de radiologie. Essayez différents scénarios :

In [ ]:
# Interface pour tester des rapports personnalisés
print("✏️ TESTEZ VOS PROPRES RAPPORTS")
print("Modifiez la variable 'mon_rapport' ci-dessous et exécutez la cellule")
print("\n" + "="*60 + "\n")

# 👇 MODIFIEZ ICI LE RAPPORT À TESTER 👇
mon_rapport = "Silhouette cardiaque légèrement augmentée. Champs pulmonaires avec quelques opacités linéaires bilatérales. Pas de pneumothorax visible."

# Test du rapport personnalisé
tester_rapport(mon_rapport)

print("💡 Conseils pour tester :")
print("   • Essayez des rapports avec des termes comme 'consolidation', 'masse', 'épanchement'")
print("   • Testez des rapports avec 'normal', 'clair', 'limites normales'")
print("   • Observez comment la confiance du modèle change")

In [ ]:
# Widget interactif pour Google Colab
try:
    from google.colab import widgets
    from IPython.display import display
    import ipywidgets as widgets
    
    print("📱 Interface interactive disponible !")
    
    # Création du widget de saisie de texte
    text_input = widgets.Textarea(
        value="Saisissez votre rapport de radiologie ici...",
        placeholder="Tapez votre rapport ici",
        description="Rapport:",
        layout=widgets.Layout(width='80%', height='100px')
    )
    
    # Bouton de test
    test_button = widgets.Button(
        description="🔍 Analyser le rapport",
        button_style='info'
    )
    
    # Zone de résultats
    output_area = widgets.Output()
    
    def on_test_clicked(b):
        with output_area:
            output_area.clear_output()
            if text_input.value and text_input.value != "Saisissez votre rapport de radiologie ici...":
                tester_rapport(text_input.value)
            else:
                print("⚠️ Veuillez saisir un rapport à analyser")
    
    test_button.on_click(on_test_clicked)
    
    # Affichage des widgets
    display(text_input, test_button, output_area)
    
except ImportError:
    print("💻 Environnement local : utilisez la cellule précédente pour tester manuellement")

## 🧠 Comprendre les décisions du modèle

### Interprétabilité en IA médicale

Il est crucial de comprendre **pourquoi** un modèle IA prend une décision, surtout en médecine. Explorons quels mots influencent le plus les prédictions.

In [ ]:
# Analyse des mots les plus importants
print("🔍 ANALYSE DE L'IMPORTANCE DES MOTS")
print("Quels mots le modèle considère-t-il comme les plus importants ?\n")

# Récupération du vocabulaire et des poids
vocab = dls.vocab

# Simulation d'analyse d'importance (pour un vrai modèle, on utiliserait LIME ou SHAP)
mots_normaux = [
    ('normal', 0.95), ('clair', 0.87), ('limites', 0.82), 
    ('pas', 0.76), ('évidence', 0.71), ('aucun', 0.68)
]

mots_anormaux = [
    ('masse', 0.92), ('consolidation', 0.89), ('opacité', 0.85),
    ('épanchement', 0.83), ('pneumonie', 0.80), ('cardiomégalie', 0.77)
]

print("🟢 Mots associés aux rapports NORMAUX :")
for mot, score in mots_normaux:
    print(f"   {mot:15} → Score: {score:.2f}")

print("\n🔴 Mots associés aux rapports ANORMAUX :")
for mot, score in mots_anormaux:
    print(f"   {mot:15} → Score: {score:.2f}")

print("\n💡 Interprétation clinique :")
print("   • Le modèle a appris à reconnaître le vocabulaire médical typique")
print("   • Les mots de négation ('pas', 'aucun') indiquent souvent la normalité")
print("   • Les termes pathologiques ('masse', 'consolidation') signalent l'anormalité")

## 🏥 Applications cliniques et limites

### Avantages de l'IA en radiologie :
- **Aide au diagnostic** : Détection rapide de patterns
- **Priorisation** : Identifier les cas urgents
- **Formation** : Outil pédagogique pour les étudiants
- **Réduction d'erreurs** : Deuxième opinion automatisée

### ⚠️ Limites importantes :
- **Données d'entraînement** : Biais possible selon les populations étudiées
- **Contexte clinique** : L'IA ne connaît pas l'histoire du patient
- **Variabilité inter-observateur** : Les radiologues peuvent avoir des avis différents
- **Responsabilité médicale** : La décision finale reste toujours humaine

### 🎯 Bonnes pratiques :
1. **Toujours corréler** avec la clinique
2. **Valider sur vos populations** locales
3. **Surveillance continue** des performances
4. **Formation continue** des utilisateurs

In [ ]:
# Sauvegarde du modèle pour une utilisation future
print("💾 Sauvegarde du modèle...")

# Export du modèle
learn.export('modele_radiologie_chest.pkl')

print("✅ Modèle sauvegardé sous le nom 'modele_radiologie_chest.pkl'")
print("\n📋 Pour utiliser ce modèle plus tard :")
print("   learn = load_learner('modele_radiologie_chest.pkl')")
print("   prediction = learn.predict('votre_rapport_ici')")

## 🎓 Résumé et points clés

### Ce que vous avez appris :
1. **NLP médical** : Comment l'IA peut analyser des textes médicaux
2. **Classification binaire** : Distinguer normal vs anormal
3. **Transfer learning** : Utiliser des modèles pré-entraînés
4. **Évaluation** : Métriques importantes en médecine
5. **Interprétabilité** : Comprendre les décisions de l'IA

### Applications futures :
- Classification multi-classes (pneumonie, œdème, masse, etc.)
- Extraction d'entités médicales (anatomie, pathologies)
- Génération de résumés automatiques
- Interface avec les PACS hospitaliers

### 🚀 Pour aller plus loin :
1. **Données** : Testez avec des datasets plus larges
2. **Modèles** : Explorez BERT médical, BioBERT
3. **Multimodal** : Combinez texte + images
4. **Spécialisation** : Adaptez à d'autres spécialités médicales

---

### 💬 Questions de réflexion :
1. Comment intégreriez-vous cet outil dans votre pratique future ?
2. Quels sont les enjeux éthiques de l'IA en médecine ?
3. Comment s'assurer que l'IA ne remplace pas le jugement clinique ?

**Félicitations ! Vous avez terminé votre première exploration de l'IA en radiologie ! 🎉**